### Lake Taupo

**Station Name:** tgho 

**Location:** North Island, New Zealand

**Archive:** [Geonet](https://www.geonet.org.nz/)

**DOI:**  	N/A

**Ellipsoidal Coordinates:**

- Latitude: -38.813

- Longitude: 175.996

- Height: -38.813 m

[Station Page at Geonet](https://www.geonet.org.nz/data/network/mark/TGHO)

[Station Page at Nevada Geodetic Laboratory](http://geodesy.unr.edu/NGLStationPages/stations/TGHO.sta)

[Google Map Link](https://goo.gl/maps/1zmgi6rRHPVPDAfV8)

<p align="center">
<img src="https://gnss-reflections.org/static/images/TGHO.jpg" width="500"><BR>
Photo credit: Simon Barker
</P>

### Data Summary

Station tgho is operated by GNS in new zeland and the site site is located 
on a platform in Lake Taupo, New Zealand. It records standard GPS and Glonass signals at a low sample rate of 30 sec.

### Imports

In [ ]:
from gnssrefl.daily_avg_cl import daily_avg
from gnssrefl.gnssir_input import make_gnssir_input
from gnssrefl.gnssir_cl import gnssir
from gnssrefl.quickLook_cl import quicklook
from gnssrefl.rinex2snr_cl import rinex2snr
from gnssrefl.installexe_cl import installexe
import ipywidgets as widgets
import os

from pathlib import Path

from gnssrefl.utils import check_environment, set_environment, get_sys

notebook_dir = Path.cwd().parents[1]

# Making sure environment variables are set - this is required to run the gnssrefl code
exists = check_environment()
if exists == False:
    set_environment(
        refl_code=str(notebook_dir),
        orbits=str(notebook_dir / "orbits"),
        exe=str(notebook_dir / "exe"),
    )

# If you are running this locally - make sure the items in the exe folder have execution permissions

# Set local variable of refl_code location
refl_code_loc = os.environ["REFL_CODE"]
print("refl_code location:", refl_code_loc)

# import gnssrefl functions


# import the crx2rnx file which is dependent on your working OS - this is required to run the gnssrefl code
# If in docker environment, then you do not need to download crxnrnx
try:
    os.environ["DOCKER"]
except KeyError:
    sys = get_sys()
    installexe(sys)

**Fast Mode:** 
this will download data weekly instead of daily

In [ ]:
weekly = widgets.Checkbox(
    value=True, description="Fast Mode", disabled=False, indent=False
)
weekly

Deselect the Fast Mode checkbox to deactivate fast mode.

### Take a Quick Look at the Data

We will begin like always by making an **SNR** file. Here we will use both GPS and Glonass and we'll set the archive to nz

In [ ]:
station = "tgho"
# picking a year and day of year
year = 2020
doy = 300

rinex2snr("tgho", year, doy, orb="gnss", archive="nz")

values, metrics = quicklook(station, year, doy)

 Using the default elevation angles is likely combining water and pier reflections so we will try restricting the close reflections (from the pier) by using a larger lower bound. Let's compare the defaults with 5-15 degrees elevation angles instead of 5-25 to see if this improves the retrievals.

In [ ]:
values, metrics = quicklook(station, year, doy=doy, e2=15)

This does look better. It's possible that the clutter near the monument produces noise at the smaller RH values. A better result can be found if those values are eliminated by setting heights from 2 through 8. (note: default e1=5 and h2=8 so we don't need to set them in the code)

In [ ]:
values, metrics = quicklook(station, year, doy, e2=15, h1=2)

We'll keep in mind from the qc plots here that we'll probably choose our amplitude to be ~9 and our peak to noise ratio to 3.

Now let's try looking at the periodograms for L2:

In [ ]:
values, metrics = quicklook(station, year, doy=doy, e1=5, e2=15, h1=2, h2=8, fr=2)

This shows us that we will not use L2 data in the analysis.

Next, we'll look at the Glonass frequencies:101 (L1) and 102 (L2)

In [ ]:
values, metrics = quicklook(station, year, doy=doy,
                            e1=5, e2=15, h1=2, h2=8, fr=101)

In [ ]:
values, metrics = quicklook(station, year, doy=doy, e1=5, e2=15, h1=2, h2=8, fr=101)

values, metrics = quicklook(station, year, doy=doy, e1=5, e2=15, h1=2, h2=8, fr=102)

We should exclude 135-225 degrees in azimuth as there is an empty region as well as a few poor retreivals.

A windy day could cause bad retrievals.  For days with no wind, we would have large amplitudes - and for very windy days it could look like this:

In [ ]:
year = 2020
doy = 273  # sep 29 was a very windy day

rinex2snr(station=station, year=year, doy=doy, archive="nz")

values, metrics = quicklook(
    station, year=year, doy=doy, e1=5, e2=15, h1=2, h2=8)

### Analyze the Data
Here we'll set the values we discussed as variables and will pass them into the `make_gnssir_input` function.
We will exclude 135-225 degrees in azimuth. We will require an amplitude of 9 and a peak to noise ratio of 3.0
and specify L1 and glonass signals (frequencies 101 and 102).

In [ ]:
min_elevation_angle = 5
max_elevation_angle = 15
peak_to_noise = 3
amplitude = 9
min_height = 2
max_height = 8
azimuths = [0, 135, 225, 380]
frequencies = [1, 101, 102]

make_gnssir_input(
    station,
    e1=min_elevation_angle,
    e2=max_elevation_angle,
    peak2noise=peak_to_noise,
    ampl=amplitude,
    h1=min_height,
    h2=max_height,
    azlist2=azimuths,
    frlist=frequencies,
)

Now we would run rinex2snr for about six months in 2020 and then run gnssir for the same dates.

In [ ]:
# get snr (let's use GFZ orbits, which is multi-GNSS setting orb='gnss')
rinex2snr(
    station, year, doy=130, doy_end=319, orb="gnss", archive="nz", weekly=weekly.value
)

Now run gnssir:


In [ ]:
gnssir(station, year, doy=130, doy_end=319)

To look at daily averages, use the utility <code>daily_avg</code>. The median filter is set to allow values within 0.25 meters of the 
median, and the minimum number of tracks required to calculate the average we will set to 50 tracks. We can also set our own name for the daily average file.

In [ ]:
daily_avg(station, medfilter=0.25, ReqTracks=50)

and now we can see the reflector heights for these 6 months in 2020.

We'll note that there are quite a few data outages in this series, which means the RINEX files were missing from the NZ archive.

Although Taupo is in a volcanic caldera, lake levels are determined by seasonal processes such 
as evaporation, precipitation, input from local drainages, and outflow. The Waikoto 
River is sole river draining the lake, and river flow is regulated by a series of hydroelectric dams.